# 0.5 x 0.5 degree monthly average MODIS EVI over Continental US

For use a covariate in mapping SIF predictions back to the data scale.

In [1]:
import sys
sys.path.insert(0, "../src")

from functools import reduce

import numpy as np
import pandas as pd 
import xarray as xr

from data_utils import GridConfig, prep_gridded_df

In [2]:
ds_evi = xr.open_dataset("../data/exp_pro/MODIS_EVI_conus.nc", engine="h5netcdf")

In [3]:
ds_evi

<xarray.Dataset>
Dimensions:  (time: 72, lon: 1400, lat: 880)
Coordinates:
  * lat      (lat) float64 61.97 61.92 61.88 61.83 ... 18.17 18.12 18.08 18.02
  * lon      (lon) float64 -130.0 -129.9 -129.9 -129.8 ... -60.12 -60.07 -60.02
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (time, lon, lat) float32 ...

In [4]:
# Computed the gridded, aggregated dataset
config = GridConfig((-125, -65, 22, 58), lon_res=0.5, lat_res=0.5)
df = prep_gridded_df(ds_evi, config)
df

,lon,lat,time,evi
0,-125.0,48.5,2015-01-01,33215384.0
1,-125.0,48.5,2015-02-01,29256924.0
2,-125.0,48.5,2015-03-01,30444616.0
3,-125.0,48.5,2015-04-01,30495000.0
4,-125.0,48.5,2015-05-01,32613076.0
...,...,...,...,...
459715,-65.0,58.0,2020-08-01,28803600.0
459716,-65.0,58.0,2020-09-01,28293100.0
459717,-65.0,58.0,2020-10-01,-1936100.0
459718,-65.0,58.0,2020-11-01,-5966900.0


In [5]:
ds = df.set_index(["lon", "lat", "time"]).to_xarray()
ds

<xarray.Dataset>
Dimensions:  (lon: 121, lat: 73, time: 72)
Coordinates:
  * lon      (lon) float64 -125.0 -124.5 -124.0 -123.5 ... -66.0 -65.5 -65.0
  * lat      (lat) float64 22.0 22.5 23.0 23.5 24.0 ... 56.0 56.5 57.0 57.5 58.0
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (lon, lat, time) float32 nan nan nan ... -5.967e+06 -8.536e+06

In [18]:
ds.to_netcdf("../data/exp_pro/MODIS_05deg_monthly_conus.nc", format="NETCDF4")
ds.close()